In [0]:
from sklearn import svm
import pandas as pd
import numpy as np
import ast
import pickle

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
data_classes = ['high_crime','low_crime','no_crime']

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/combined_pred.csv")
rekog_preds = df['rekog_pred']
recok_image_id = df['image_id']
y_true_lable = df['y_true']
rekog_prob = df['rekog_prob'].apply(ast.literal_eval)
df.head()

,image_id,y_true,rekog_pred,rekog_prob
0,a1f0fd29e6ae9a8618baac0a27e13676e192ee03c38e79...,HighCrime,HighCrime,"[0.9994573468578152, 5.315951564895016e-09, 0...."
1,68934fb5c279c38cda8e88fa67d2d6a664ec3fefac0793...,NoCrime,NoCrime,"[0.003945948949745179, 0.026098010084206702, 0..."
2,73e74e658d83ec20d35cf16ce461b9538ecdc59ea70ff8...,NoCrime,NoCrime,"[0.01228890124602555, 1.1865378451731604e-06, ..."
3,676508fcd73d674bac82600e02b5d58a562a7493046baf...,NoCrime,NoCrime,"[0.024695431414419152, 0.0011542972092025909, ..."
4,7a6c29f171fe6d3b550f24d7650c229e298c67fc2bbef3...,NoCrime,NoCrime,"[0.00011153877314141146, 1.3805475472947983e-0..."


In [0]:
rekog_pred_val = []
for rekog_pred in rekog_preds:
    rekog_pred_val.append(rekog_pred.split())

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

one_hot = MultiLabelBinarizer()
y_pred_rekog = one_hot.fit_transform(rekog_pred_val)


In [0]:
image_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/fastai_resnet18_70_training_result.csv')
image_df = image_df[pd.notnull(image_df['object_labels'])]
y_pred = image_df['y_pred'].apply(ast.literal_eval)
y_true = image_df['y_true'].apply(ast.literal_eval)
y_pred_prob = image_df['y_pred_prob'].apply(ast.literal_eval)

image_df.head()

,id,y_true,y_pred,y_pred_prob,object_labels
0,a1f0fd29e6ae9a8618baac0a27e13676e192ee03c38e79...,"[1, 0, 0]","[1.0, 0.0, 0.0]","[0.8746466, 0.02755347, 0.08695286]","Human,Person,Gun,Weapon,Weaponry,Military,Mili..."
1,68934fb5c279c38cda8e88fa67d2d6a664ec3fefac0793...,"[0, 0, 1]","[0.0, 0.0, 1.0]","[0.03033742, 0.06771197, 0.8814333]","Human,Person,Art,Automobile,Transportation,Veh..."
2,73e74e658d83ec20d35cf16ce461b9538ecdc59ea70ff8...,"[0, 0, 1]","[0.0, 0.0, 1.0]","[0.1415521, 0.08210678, 0.7961538]","Plot,Text,Diagram,Number,Symbol,Page,Document,..."
3,676508fcd73d674bac82600e02b5d58a562a7493046baf...,"[0, 0, 1]","[0.0, 0.0, 1.0]","[0.02073421, 0.004477706, 0.9702271]","Text,Poster,Advertisement,Handwriting,Word"
4,7a6c29f171fe6d3b550f24d7650c229e298c67fc2bbef3...,"[0, 0, 1]","[0.0, 0.0, 1.0]","[0.03509314, 0.00011819219, 0.98366]","Human,Person,Face,Crowd,Audience,Speech,Finger..."


In [0]:
y_pred_image = [list(map(int,i)) for i in y_pred]

In [0]:
len(y_pred_prob)

1971

In [0]:
train_x = []

for i,values in enumerate(y_pred_image):
  combined = np.vstack((values, y_pred_rekog[i])).T
  train_x.append(combined)

In [0]:
for test in rekog_prob:
  if (np.array(test).shape[0]) is not 3:
    print('failed')

In [0]:
rekog_prob.shape

(1971,)

In [0]:
train_x_prob = []

for i,values in enumerate(y_pred_prob):
  combined = np.vstack((values, rekog_prob[i])).T
  train_x_prob.append(combined)

In [0]:
train_x = np.array(train_x)

In [0]:
train_x_prob = np.array(train_x_prob)

In [0]:
train_x.shape

(1971, 3, 2)

In [0]:
nsamples, nx, ny = train_x.shape
d2_train_dataset = train_x.reshape((nsamples,nx*ny))

In [0]:
nsamples, nx, ny = train_x_prob.shape
d2_train_dataset_prob = train_x_prob.reshape((nsamples,nx*ny))

In [0]:
d2_train_dataset.shape

(1971, 6)

In [0]:
clf = svm.SVC(probability=True)

# clf.fit(X, y) 
clf.fit(d2_train_dataset_prob, np.array(y_true_lable)) 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
filename = '/content/gdrive/My Drive/Colab Notebooks/svm_prob.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier

clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))



In [0]:
clf.fit(d2_train_dataset, np.array(y_true_lable)) 

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=4, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=-1, nthread=None,
                                            objective='binary:logistic',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            verbosity=1),
                    n_jobs=None)

In [0]:
clf.predict_proba([[0,1,0,0,0,1]])

array([[0.63350904, 0.13089555, 0.23559541]])